In [1]:
import configparser

config = configparser.ConfigParser()
config.read('Input/YbB12_resistivity_leads.log')

['Input/YbB12_resistivity_leads.log']

In [2]:
print(dict(config['Acquisition']))

{'data directory': 'E:\\Other Group\\McQueen\\24102019', 'filename prefix': 'YbB12_resistivity_leads', 'configuration': 'C:\\SkyScan\\config\\Std_60_80_100kV_', 'number of files': '202', 'source voltage (kv)': '100', 'source current (ua)': '100', 'number of rows': '1332', 'number of columns': '2000', 'image crop origin x': '0', 'image crop origin y': '0', 'camera binning': '2x2', 'image rotation': '-0.1450', 'gantry direction': 'CC', 'image pixel size (um)': '9.87', 'object to source (mm)': '116.370', 'camera to source (mm)': '210.131', 'vertical object position (mm)': '45.000', 'optical axis (line)': '657', 'filter': 'Al+Cu', 'image format': 'TIFF', 'depth (bits)': '16', 'screen lut': '0', 'exposure (ms)': '1360', 'rotation step (deg)': '1.000', 'frame averaging': 'ON (1)', 'random movement': 'OFF (5)', 'use 360 rotation': 'NO', 'geometrical correction': 'ON', 'camera offset': 'OFF', 'median filtering': 'ON', 'flat field correction': 'ON', 'rotation direction': 'CC', 'scanning traject

In [3]:
theta_step = float(config['Acquisition']['rotation step (deg)'])

In [4]:
from pathlib import Path
import numpy as np
sd = Path('Input')
prefix = config['Acquisition']['filename prefix']
tiffs = sorted(sd.glob(prefix + "[0-9]*.tif"))
num_files = len(tiffs)
ind = list(range(0,num_files))
print(ind)
theta = np.array(ind)*theta_step/180*np.pi;
print(theta)
print(tiffs[0])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200]
[0.         0.01745329 0.03490659 0.05235988 0.06981317 0.08726646
 0.10471976 0.12217305 0.13962634 0.1

In [5]:
import dxchange.reader as dxreader
tomo = dxreader.read_tiff_stack(tiffs[0],ind)

In [6]:
minval = np.amin(tomo)
maxval = np.amax(tomo)
tomo = (tomo-minval)/(maxval-minval)

In [7]:
import logging
logging.basicConfig(level=logging.INFO)
import tomopy
rot_center = tomopy.find_center(tomo, theta)

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:tomopy.recon.rotation:Trying rotation center: [1000.]
INFO:tomopy.recon.rotation:Function value = 0.597278
INFO:tomopy.recon.rotation:Trying rotation center: [1050.]
INFO:tomopy.recon.rotation:Function value = 0.716903
INFO:tomopy.recon.rotation:Trying rotation center: [950.]
INFO:tomopy.recon.rotation:Function value = 0.717496
INFO:tomopy.recon.rotation:Trying rotation center: [1025.]
INFO:tomopy.recon.rotation:Function value = 0.624130
INFO:tomopy.recon.rotation:Trying rotation center: [975.]
INFO:tomopy.recon.rotation:Function value = 0.622509
INFO:tomopy.recon.rotation:Trying rotation center: [987.5]
INFO:tomopy.recon.rotation:Function value = 0.588916
INFO:tomopy.recon.rotation:Trying rotation center: [975.]
INFO:tomopy.recon.rotation:Function value = 0.622509
INFO:tomopy.recon.rotation:Trying rotation center: 

In [8]:
print(rot_center)

[991.40625]


In [9]:
proj = tomopy.minus_log(tomo)

In [10]:
recon = tomopy.recon(proj, theta, center=rot_center, algorithm='gridrec', sinogram_order=False)

In [11]:
import ipyvolume as ipv
ipv.quickvolshow(recon)

/home/idies/miniconda3/envs/py37/lib/python3.7/site-packages/ipyvolume/widgets.py:241: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data_view = self.data_original[view]
/home/idies/miniconda3/envs/py37/lib/python3.7/site-packages/ipyvolume/utils.py:207: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data = (data[slices1] + data[slices2]) / 2


In [12]:
recon2 = np.copy(recon)
recon2 = tomopy.circ_mask(recon2, axis=0, ratio=0.95)

In [13]:
ipv.quickvolshow(recon2)

/home/idies/miniconda3/envs/py37/lib/python3.7/site-packages/ipyvolume/serialize.py:92: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0] ** 2 + gradient[1] ** 2 + gradient[2] ** 2)
